# İkinci yol ise ;

* Küçük bir veri setini elle etiketleyip YOLO ile fine-tuning. Adım adım yapacağımız plan şöyle olacak:

## 1️⃣ Küçük Dataseti Seçmek

* Önce her sınıftan (bump ve road) 50–100 görsel seçebilirsin.

* Bu küçük set YOLO modelini eğitmek için yeterli olacak.

Örn:

```bash 
train/
  bump/   -> 50 görsel
  road/   -> 50 görsel
val/
  bump/   -> 10–20 görsel
  road/   -> 10–20 görsel


## Şimdi diğer yolda denediğimiz bounding box kordinatlarını silelim

In [1]:
import os
from pathlib import Path

dataset_path = Path(r"C:\YOLO\YOLO\Uygulamalar\Dataset\Speed_Bump_Dataset\bump_detection_dataset_yolo")
splits = ["train", "test"]

for split in splits:
    lbl_dir = dataset_path / split / "labels"
    for txt_file in lbl_dir.glob("*.txt"):
        txt_file.unlink()  # dosyayı sil
        print(f"Deleted: {txt_file.name}")


Deleted: Augusta9c.txt
Deleted: B4321.txt
Deleted: bmw-2-series-does-savage-highway-drifting-in-turkey-goes-all-wide_1.txt
Deleted: bmw-2-series-does-savage-highway-drifting-in-turkey-goes-all-wide_2.txt
Deleted: bottom-parsons-hill-streetview.txt
Deleted: BPP_MDG_080817speed_4683JPG.txt
Deleted: broad-walk-streetview.txt
Deleted: fig206.txt
Deleted: GettyImages-135373383-Cropped-2.txt
Deleted: hqdefault.txt
Deleted: Image00059.txt
Deleted: Image00060.txt
Deleted: Image00061.txt
Deleted: Image00062.txt
Deleted: Image00063.txt
Deleted: Image00064.txt
Deleted: Image00065.txt
Deleted: Image00067.txt
Deleted: Image00068.txt
Deleted: Image00069.txt
Deleted: Image00070.txt
Deleted: Image00071.txt
Deleted: Image00072.txt
Deleted: Image00073.txt
Deleted: Image00074.txt
Deleted: Image00075.txt
Deleted: Image00076.txt
Deleted: Image00077.txt
Deleted: Image00078.txt
Deleted: Image00080.txt
Deleted: Image00081.txt
Deleted: Image00082.txt
Deleted: Image00083.txt
Deleted: Image00084.txt
Deleted: Ima

# Şimdi ise farklı bir klasöre train ve testten 50 - 20 olacak şekilde veriler koyalım ve bunları yine label olarak etiketleyelim.

### Yani klasör yapımızı şöyle yapalım :

```bash 
bump_detection_dataset_yolo_small/
├── train/
│   ├── images/
│   └── labels/
└── val/
    ├── images/
    └── labels/


### Fine-Tuning için Hazırlık

* data.yaml dosyasını küçük dataset için güncelle:

```bash 
train: bump_detection_dataset_yolo_small/train/images
val: bump_detection_dataset_yolo_small/val/images

nc: 2
names: ['bump', 'road']



Bu uygulamayı diğer notebooklarda anlattığım Label-Img ile yapabilirsiniz.Sonrasında bu etiketlemelere göre YOLO diğerlerinden çıkarım yapacaktır.
----
----

## Etiketleme işlemi bittikden sonra class_idlere göz gezdirmenizi istiyorum.Mesela şu an bence bulunan bir .txt formatının içeriği şu şekilde ;

* 16 0.198198 0.700441 0.360360 0.264317

### Bu ifade yanlış çünkü ben .yaml uzantımda bu değerlerin class adlarını 0 ve 1 olarak verdim.Ve şimdi bu sorunu minik bir script ile çözeceğiz.Sizde böyle bir hata yoksa burayı atlayabilirsiniz.

In [1]:
from pathlib import Path

labels_dir = Path(r"C:\YOLO\YOLO\Uygulamalar\Dataset\Dataset\Train\labels")

# Class mapping: eski_id → yeni_id
class_map = {
    16: 0,  # bump
    15: 1   # road
}

for txt_file in labels_dir.glob("*.txt"):
    lines = []
    with open(txt_file, "r") as f:
        for line in f.readlines():
            parts = line.strip().split()
            old_id = int(parts[0])
            if old_id in class_map:
                parts[0] = str(class_map[old_id])
            lines.append(" ".join(parts))
    with open(txt_file, "w") as f:
        f.write("\n".join(lines))
    print(f"Updated: {txt_file.name}")


Updated: Augusta9c.txt
Updated: B4321.txt
Updated: bmw-2-series-does-savage-highway-drifting-in-turkey-goes-all-wide_1.txt
Updated: bmw-2-series-does-savage-highway-drifting-in-turkey-goes-all-wide_2.txt
Updated: bottom-parsons-hill-streetview.txt
Updated: BPP_MDG_080817speed_4683JPG.txt
Updated: broad-walk-streetview.txt
Updated: Image00059.txt
Updated: Image00060.txt
Updated: Image00061.txt
Updated: Image00063.txt
Updated: Image00064.txt
Updated: Image00065.txt
Updated: Image00068.txt
Updated: Image00069.txt
Updated: Image00072.txt
Updated: Image00073.txt
Updated: Image00076.txt
Updated: Image00077.txt
Updated: Image00082.txt
Updated: Image00083.txt
Updated: Image00084.txt
Updated: Image00085.txt
Updated: Image00086.txt
Updated: Image00087.txt
Updated: Image00089.txt
Updated: Image00096.txt
Updated: Image00097.txt
Updated: Image00098.txt
Updated: Image00099.txt
Updated: Image00100.txt
Updated: Image00101.txt
Updated: Image00102.txt
Updated: image59-1.txt
Updated: image59.txt
Updated:

In [2]:
from pathlib import Path

labels_dir = Path(r"C:\YOLO\YOLO\Uygulamalar\Dataset\Dataset\Test\labels")

# Class mapping: eski_id → yeni_id
class_map = {
    16: 0,  # bump
    15: 1   # road
}

for txt_file in labels_dir.glob("*.txt"):
    lines = []
    with open(txt_file, "r") as f:
        for line in f.readlines():
            parts = line.strip().split()
            old_id = int(parts[0])
            if old_id in class_map:
                parts[0] = str(class_map[old_id])
            lines.append(" ".join(parts))
    with open(txt_file, "w") as f:
        f.write("\n".join(lines))
    print(f"Updated: {txt_file.name}")


Updated: 135dzlC5HyQna0uZlLbvAXA.txt
Updated: 15.txt
Updated: 240_F_203017289_GyGncBbG9c31PmG2GAc61nQv0S5rfVhl.txt
Updated: 240_F_212697340_TCCsAMQEm6XeBls4swjmylGhqHcOXmfq.txt
Updated: 246059_07203f5a2b86416692725d87160a08a7mv2.txt
Updated: 281b2212.txt
Updated: 4400250_d3ca5b09.txt
Updated: 4826434_22b027c6.txt
Updated: 5005881b4cc75.image.txt
Updated: 500_F_74497238_LnrFH7zCOgi8tpzfpMn58uTVXWKWEhCQ.txt
Updated: 5024891_bbc107cc.txt
Updated: 51g42uL6EIL._SX425_.txt
Updated: 5589832_4abe38c6.txt
Updated: 72782d6b59b202d03a060867607246e7de4be80f153fc6ee888a9327dfd076ad.txt
Updated: images52.txt
Updated: images68.txt
Updated: IMG_2108 3.txt
Updated: IMG_2109 0009.txt
Updated: IMG_2109 0010.txt
Updated: IMG_2109 0023.txt
Updated: IMG_2109 0105.txt
Updated: IMG_2109 0106.txt
Updated: IMG_2109 0107.txt
Updated: IMG_2109 0117.txt
Updated: IMG_2109 0155.txt
Updated: IMG_2109 0165.txt


----

# Bir sonraki adım: Fine-Tuning

* Küçük dataset üzerinde pretrained YOLOv8 modeli ile fine-tuning yapabiliriz.

🔹 Açıklamalar:

* epochs=100: Daha uzun eğitim, modelin ince detayları öğrenmesini sağlar.

* augment=True: Flip, rotate, color jitter gibi augmentasyonları aktif eder, modelin genellemesi artar.

* lrf=0.2: Learning rate yavaş yavaş azalacak, aşırı öğrenmeyi önler.

* patience=15: Eğer model belirli sayıda epoch iyileşme göstermezse eğitim durur.

* augment=True sayesinde Mosaic ve MixUp otomatik aktif olacak.

In [20]:
from ultralytics import YOLO

# Önceki fine-tuned model
model = YOLO("runs/detect/train/weights/best.pt")

# Optimize edilmiş ileri düzey fine-tuning, bump ağırlıklı
results = model.train(
    data=r"C:\YOLO\YOLO\Uygulamalar\Dataset\Dataset\data.yaml",
    epochs=80,                # uzun eğitim
    batch=16,
    imgsz=[640, 704, 768],    # multi-scale training
    lr0=0.001,
    lrf=0.2,
    momentum=0.937,
    weight_decay=0.0005,
    optimizer="SGD",
    augment=True,             # Mosaic, MixUp, RandAugment aktif
    patience=15,
    project="runs/train_finetune_max",
    name="road_bump_weighted",
    exist_ok=True,
    freeze=[0],               # backbone'i önce dondur
    cache=True,
    plots=True,
    verbose=True # bump=2.0, road=1.0
)

Ultralytics 8.3.203  Python-3.13.5 torch-2.8.0+cpu CPU (AMD Ryzen 7 5800H with Radeon Graphics)
engine\trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\YOLO\YOLO\Uygulamalar\Dataset\Dataset\data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=80, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=[0], half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=[640, 704, 768], int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.2, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=runs/detect/train/weights/best.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=road_bump_weighted, nbs=64, nms=False, opset=None, optimize=False, optimi

## Bu tablo her epoch sonunda güncellenecek ve şunları göreceksin:

* box_loss → bounding box tahmini hatası

* cls_loss → class tahmin hatası

* dfl_loss → dağılım odaklı loss (YOLOv8 spesifik)

* Instances → batch içindeki objelerin sayısı

* Size → eğitim sırasında kullanılan resim boyutu

----

# Yoloya verileri tanıttıkdan sonraki işlem bu veriler için yine bir klasör oluşturmak.Normalde bu yapıları yapmamıza gerek yok fakat daha doğru bir çözüm ve daha az karışıklık için öneriliyor.Bu işlemler sonucunda aşağıdaki gibi bir yapı bizleri karşılayacak

```bash
Remaining/
│
├── Train/
│   ├── images/
│   └── labels/   # başlangıçta boş olabilir
│
└── Test/
    ├── images/
    └── labels/   # başlangıçta boş olabilir


* images → otomatik etiketlenecek görseller

* labels → YOLO predict çalıştırınca otomatik oluşturulacak .txt dosyaları buraya kaydolacak

```bash 
Remaining/
│
├── Train/
│   ├── images/     ← kopyaladığın görseller
│   └── labels/     ← otomatik etiketler buraya gelecek
│
└── Test/
    ├── images/     ← kopyaladığın görseller
    └── labels/     ← otomatik etiketler buraya gelecek


----
# Sonraki Adım: Otomatik Labeling - Train Ve Test -

* Test için:

In [24]:
from ultralytics import YOLO
from pathlib import Path

# Fine-tuned model
model_path = r"C:\YOLO\YOLO\Uygulamalar\Uygulama - 1 (Yol - Tümsek)\runs\detect\train\weights\best.pt"
model = YOLO(model_path)

# TEST görselleri
test_images = r"C:\YOLO\YOLO\Uygulamalar\Dataset\Remaining\Test\images"

# Çıktı klasörü
test_output = Path(r"C:\YOLO\YOLO\Uygulamalar\Dataset\Remaining\Test\predictions")

# TEST tahmin
print("Processing TEST images...")
test_results = model.predict(
    source=test_images,
    save=True,              # Tahmin görselleri kaydedilsin
    save_txt=True,          # Bounding box .txt kaydedilsin
    project=str(test_output.parent),
    name="predictions",
    exist_ok=True,
    imgsz=640,
    conf=0.2,
    iou=0.3,
    augment=True,
    verbose=True,
    batch=16
)

print(f"TEST completed. Results saved in {test_output}")


Processing TEST images...

image 1/2466 C:\YOLO\YOLO\Uygulamalar\Dataset\Remaining\Test\images\0425Manly_Web.jpg: 640x640 1 road, 210.1ms
image 2/2466 C:\YOLO\YOLO\Uygulamalar\Dataset\Remaining\Test\images\06jb1284_2.jpg: 640x640 1 bump, 2 roads, 210.1ms
image 3/2466 C:\YOLO\YOLO\Uygulamalar\Dataset\Remaining\Test\images\06jb1402.jpg: 640x640 1 road, 210.1ms
image 4/2466 C:\YOLO\YOLO\Uygulamalar\Dataset\Remaining\Test\images\06jbpshya_375x375.jpg: 640x640 (no detections), 210.1ms
image 5/2466 C:\YOLO\YOLO\Uygulamalar\Dataset\Remaining\Test\images\0_Marie-McGiveron-is-unhappy-with-speed-bumps-on-Queens-RoadEvertonthat-start-at-the-new-private-hou.jpg: 640x640 2 roads, 210.1ms
image 6/2466 C:\YOLO\YOLO\Uygulamalar\Dataset\Remaining\Test\images\0_Oxford-RoadJPG.jpg: 640x640 2 roads, 210.1ms
image 7/2466 C:\YOLO\YOLO\Uygulamalar\Dataset\Remaining\Test\images\0_bumps-1.jpg: 640x640 1 road, 210.1ms
image 8/2466 C:\YOLO\YOLO\Uygulamalar\Dataset\Remaining\Test\images\1.jpg: 640x640 1 bump, 3 r

* Bu adım tamamlandığında tüm dataset otomatik olarak YOLO formatında etiketlenmiş olacak

* Sonrasında bu veriyi CNN modele sokacağız